In [2]:
import numpy as np; import pandas as pd; import pyodbc; import datetime; import sklearn; import os; import math
from sklearn import feature_selection, linear_model, grid_search, cross_validation, metrics, ensemble, learning_curve 
from sklearn.preprocessing import scale
import seaborn as sns
from sklearn.manifold import TSNE
import pickle
import xgboost as xgb
from sklearn.feature_selection import chi2, f_classif, SelectKBest
from sklearn.preprocessing import StandardScaler
%pylab inline 
import sys
import warnings
conn = pyodbc.connect(r'DRIVER={SQL Server Native Client 11.0};'r'SERVER=WIN-5G5AUOCEJPK;'r'DATABASE=Gold585;'r'Trusted_Connection=yes;')

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
from IPython.core.display import display,HTML

display(HTML("<style>.container{width:90% !important;}</style>"))

Populating the interactive namespace from numpy and matplotlib


In [3]:
sql_query = "SELECT [Code] \
      ,[PartnerID] \
      ,IIf([Sex]='M',1,IIf([Sex]='F',0,NULL)) Sex \
      ,[Age] \
      ,[LombardRating] \
      ,[LombardCategoryID] \
      ,[AmountZB] \
      ,[TermWoutGracePeriod] \
      ,[WeightGross] \
      ,[WeightNet] \
      ,[WeightNet999] \
      ,[WeightNotBrilliant] \
      ,[WeightNet999Calc] \
      ,[Cost] \
      ,[CostMetal] \
      ,[CostPerGram] \
      ,[PromotionID] \
      ,[CostAmount] \
      ,[PercentPerDay] \
      ,[isGoodZB] \
      ,[isBadZB] \
      ,[BudetSledVisit] \
      ,[BudetPerezalog] \
  FROM [Gold585].[dbo].[CommonTable] \
  Where IssueDate < convert(datetime,'2018-06-01') and Category in ('Новая cсуда')"
data = pd.read_sql(sql_query,conn)

In [4]:
data.head()

,Code,PartnerID,Sex,Age,LombardRating,LombardCategoryID,AmountZB,TermWoutGracePeriod,WeightGross,WeightNet,...,Cost,CostMetal,CostPerGram,PromotionID,CostAmount,PercentPerDay,isGoodZB,isBadZB,BudetSledVisit,BudetPerezalog
0,БЛ431059,97728,0.0,46,0,12,7600.0,60,5.70,5.70,...,7619.000000,6418.600098,1336.666626,0,7600.0,0.30,0,1,0,0
1,930000077660,97729,0.0,50,0,12,400.0,60,0.66,0.51,...,387.600006,387.600006,587.272705,0,400.0,0.25,1,0,1,0
2,930000143357,97729,0.0,50,0,71,1760.0,60,1.77,1.75,...,1592.500000,1592.500000,899.717529,0,1760.0,0.30,1,0,1,0
3,930000606914,97729,0.0,50,0,12,5150.0,60,5.51,5.43,...,5158.500000,5158.500000,936.206787,0,5150.0,0.30,1,0,1,0
4,930002411764,97729,0.0,52,0,63,2650.0,60,2.53,2.53,...,2656.000000,2656.000000,1049.802246,0,2650.0,0.34,1,0,1,0


In [10]:
target = data.drop(columns=['BudetPerezalog','BudetSledVisit','isBadZB', 'isGoodZB','Code','PartnerID'],axis =1).fillna(0)
lables = data['isGoodZB']

In [11]:
X = StandardScaler().fit_transform(target)

In [ ]:
from sklearn.cluster import DBSCAN
db = DBSCAN(min_samples=10000, n_jobs = -1).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

print('Estimated number of clusters: %d' % n_clusters_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(labels_true, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

# #############################################################################
# Plot result
import matplotlib.pyplot as plt

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

In [6]:
from scipy.cluster.hierarchy import linkage, dendrogram

In [13]:
samples = X
 
# Реализация иерархической кластеризации при помощи функции linkage
mergings = linkage(samples, method='complete')
 
# Строим дендрограмму, указав параметры удобные для отображения
dendrogram(mergings,
           labels=varieties,
           leaf_rotation=90,
           leaf_font_size=6,
           )
 
plt.show()

MemoryError: 